# COVID-19 Deaths in African-American Communities

Using data from the [COVID Tracking Project](https://covidtracking.com/about-data).

To see plots for COVID-19 infections instead of plots for COVID-19 fatalities, you can refer to this [companion notebook](https://www.kaggle.com/paultimothymooney/explore-covid-19-deaths-in-african-american-commun). 

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
race_data = pd.read_csv('/kaggle/input/covid-tracking-project-racial-data-tracker/Race Data Entry - CRDT.csv').fillna('0')
todays_date = race_data['Date'].max() # Find the most recent date in the dataset
race_data = race_data[race_data.Date.isin([todays_date])] # Limit to latest data only

race_data_simple = race_data[['Deaths_Total', 'Deaths_White',
       'Deaths_Black', 'Deaths_LatinX', 'Deaths_Asian', 'Deaths_AIAN',
       'Deaths_NHPI', 'Deaths_Multiracial', 'Deaths_Other',
       'Deaths_Unknown', 'Deaths_Ethnicity_Hispanic',
       'Deaths_Ethnicity_NonHispanic', 'Deaths_Ethnicity_Unknown']].astype(float)
race_data_simple = race_data_simple.div(race_data_simple.Deaths_Total,axis=0)*100 # Calculate Percent of Total
race_data_simple = race_data_simple.fillna('0')
race_data_simple['State']= race_data['State']
race_data = race_data_simple
#race_data.to_csv('/kaggle/working/black_covid_data.csv',index=False)

In [3]:
fig = px.choropleth(race_data, 
                    locations="State", 
                    color="Deaths_Black", 
                    locationmode = 'USA-states', 
                    hover_name="Deaths_Black",
                    range_color=[10,70],
                    scope="usa",
                    title='Percentage of Total COVID-19 Deaths by Black Communities')
fig.show()

In [4]:
fig = px.bar(race_data.sort_values('State', ascending=True), 
             x="State", 
             y="Deaths_Black",
             title='Percentage of Total COVID-19 Deaths by Black Communities (alphabetical)')
fig.show()


In [5]:
df = pd.read_csv('/kaggle/input/percent-black-population-for-every-state-in-usa/percent_black_over_time.csv')
df = df[1:]
cols_to_check = ['1790', '1800', '1810', '1820', '1830', '1840',
       '1850', '1860', '1870', '1880', '1890', '1900', '1910', '1920',
       '1930', '1940', '1950', '1960', '1970', '1980', '1990', '2000',
       '2010', '2018']
df[cols_to_check] = df[cols_to_check].replace({'%':''}, regex=True)
df = df.transpose()
new_header = df.iloc[0]
df = df[1:].fillna(0)
df.columns = new_header
df = df.astype(float)
df = df[-1:]
df = df.transpose()
df.columns = ['Percent Black']
df['State/Territory'] = df.index
df.to_csv('/kaggle/working/percent_black_state_by_state.csv',index=False) # save to notebook output

In [6]:
plot = px.bar(df, x=df.index, y="Percent Black", hover_name="Percent Black",title='Percentage of Population in USA that Identifies as Black (in 2018)') 
plot

In [7]:
df = pd.read_csv('/kaggle/input/percent-black-population-for-every-state-in-usa/percent_black_in_2018.csv')
fig = px.choropleth(df, 
                    locations="State Code", 
                    color="Percent Black", 
                    locationmode = 'USA-states', 
                    hover_name="State",
                    range_color=[0,30],scope="usa",
                    title='Percent of Population that Identifies as Black (2018)')
fig.show()

In [8]:
race_data['Deaths_Black'] = race_data['Deaths_Black'].astype(float)
fig = px.bar(race_data.sort_values('Deaths_Black',ascending=False), 
             x="State", 
             y="Deaths_Black",
             title='Percentage of Total COVID-19 Deaths by Black Communities (sorted)')
fig.show()


In [9]:
race_data['Percent_Black'] = df['Percent Black']
race_data_simple = race_data[['Deaths_Black','Percent_Black']].astype(float)
relative_to_population = race_data_simple['Deaths_Black'].div(race_data_simple.Percent_Black,axis=0) # Calculate Percent of Total
relative_to_population = relative_to_population.fillna('0')
race_data['Black Deaths Relative to Population'] = relative_to_population
race_data['State Code'] = df['State Code']
race_data['State'] = df['State']
race_data.to_csv('/kaggle/working/covid_deaths_by_race.csv',index=False) # save to notebook output

In [10]:
race_data['Black Deaths Relative to Population'] = race_data['Black Deaths Relative to Population'].astype(float)
fig = px.bar(race_data.sort_values('Black Deaths Relative to Population',ascending=False), 
             x="State", 
             y="Black Deaths Relative to Population",
             title='Percent of COVID-19 Deaths vs Percent of Population (sorted)')
fig.show()


In [11]:
fig = px.choropleth(race_data, 
                    locations="State Code", 
                    color="Black Deaths Relative to Population", 
                    locationmode = 'USA-states', 
                    hover_name="State",
                    range_color=[0,3],scope="usa",
                    title='Percent of COVID-19 Deaths vs Percent of Population')
fig.show()